## **Binary Search**
A search algorithm that, given a sorted list and a target value, repeatedly divides a sorted list in half,
narrowing down the possible location of a target value until the value is found (or the range is empty).It returns the index of the target (or `None` if not found).

#### **Use Case**
Binary search is used when you need to efficiently find an element in a **sorted** list or array.
It’s ideal for large datasets where a linear scan would be too slow, as it reduces the search space by half each step.

#### **Algorithm Steps**
1. Define the search range (`left` to `right`).
2. Compute the midpoint of the range.
3. Compare the midpoint value with the target.
    - If target == midpoint → return the index.
    - If target < midpoint  → search the left half.
    - If target > midpoint  → search the right half.
4. Repeat (steps 2-3) until the range is 1 element / it's not found.

#### **Complexity**
| Type | Time | Space |
|------|------|--------|
| Iterative | O(log n) | O(1) |
| Recursive | O(log n) | O(log n) |

##### **Explanation**
**Time Complexity:** For a given list of size n, every iteration reduces the search range in half. 
This gives the algorithm at most log_2(n) (or log n) iterations.
Each of these iterations has a constant amount of operations x, totaling (log n) * x operations - or just log n for big-O notation.

**Space Complexity:** 
* *Iterative:* O(1). No data structures dependant on size were used so space is just constant
* *Recursive:* O(log n). Have to look at auxiliary space and call stack space. In this case, auxiliary space (temporary space like variables) is just left, right, mid and val. So, it's O(1).
However, call stack space, or recursion depth, is O(log n), as it splits the range in half, keeping the previous calls, until it finds the answer. In total, it's auxiliary * call stack = O(1) * O(log n) = O(log n)


#### **Implementation**

In [24]:
# Import relevant modules

from random import randint, sample
from typing import Optional

##### **Iterative version**

In [25]:
def binary_search_iter(ls: list, target: int) -> Optional[int]:
    # Define search range
    left, right = 0, len(ls) - 1 

    while left <= right:
        mid = (left + right) // 2 # For languages where overflow can be an issue, can use left + (right - left) // 2
        val = ls[mid]

        # Return midpoint index if it's the target
        if target == val:
            return mid
        
        # Otherwise, adjust search range as needed
        if target < val:
            right = mid - 1  # Don't include mid value as that's already checked
        else:
            left = mid + 1

    return None

##### **Recursive version**

In [26]:
# Recursive implementation of binary search
def binary_search_rec(ls: list, target: int) -> Optional[int]:
    def search(left: int, right: int) -> Optional[int]:
        # Base case for when it has looked through the whole range
        if left > right:
            return None
        
        mid = (left + right) // 2
        val = ls[mid]

        # Return midpoint index if it's the target
        if target == val:
            return mid
        
        # Otherwise, recursively search for value in the relevant range
        if target < val:
            idx = search(left, mid - 1)
        else:
            idx = search(mid + 1, right)

        return idx

    return search(0, len(ls) - 1)

#### **Example use**

In [27]:
if __name__ == "__main__":
    nums = sorted(sample(range(1000), 100))         # Sample of 100 random, different numbers 
    target = nums[randint(0, 99)]                   # Pick a random target that's found on the list
    idx_i = binary_search_iter(nums, target)
    idx_r = binary_search_rec(nums, target)
    for idx in [idx_i, idx_r]:
        if idx is not None:
            print(f'Target {target} found at index {idx}: nums[{idx}] = {nums[idx]}')
        else:
            print(f'Target {target} is not on the given list.')
    

Target 638 found at index 65: nums[65] = 638
Target 638 found at index 65: nums[65] = 638
